In [1]:
import time
import os
import torch
from torchvision import transforms
import random
import numpy as np
import yaml
from PIL import Image
from experiment import VAEXperiment
from models import *


import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete



from stable_baselines3 import SAC, PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

seed = 42

### VAE TRAIN

In [2]:
!python run.py -c configs/bces_vae.yaml

Global seed set to 1265
AAA Data/lunar-lander/data_comb_third_vae
MyDataset:  69615  images found
AAA Data/lunar-lander/data_comb_third_vae
MyDataset:  69615  images found
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
======= Training MSSIMVAE =======
Global seed set to 1265
initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

/home/max/miniconda3/envs/vae/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA

Epoch 3:  75%|████▌ | 1632/2176 [00:28<00:09, 58.06it/s, loss=2.08e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 4:  75%|████▌ | 1632/2176 [00:27<00:09, 58.91it/s, loss=2.09e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 5:  75%|████▌ | 1632/2176 [00:27<00:09, 58.42it/s, loss=2.08e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 5:  80%|████▊ | 1746/2176 [00:29<00:07, 58.54it/s, loss=2.08e+03, v_num=8]


Epoch 6:  75%|████▌ | 1632/2176 [00:28<00:09, 56.76it/s, loss=2.08e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 7:  75%|████▌ | 1632/2176 [00:27<00:09, 59.13it/s, loss=2.09e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Validating:  97%|████████████████████████████▏| 528/544 [00:07<00:00, 69.32it/s]


Epoch 8:  75%|████▌ | 1632/2176 [00:28<00:09, 58.05it/s, loss=2.28e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 9:  75%|████▌ | 1632/2176 [00:27<00:09, 58.82it/s, loss=2.07e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 10:  75%|███▊ | 1632/2176 [00:29<00:09, 56.18it/s, loss=2.06e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Validating:  72%|████████████████████▋        | 389/544 [00:05<00:02, 64.83it/s]


Epoch 11:  75%|███▊ | 1632/2176 [00:27<00:09, 59.86it/s, loss=2.06e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 12:  75%|███▊ | 1632/2176 [00:28<00:09, 57.75it/s, loss=2.11e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 13:  75%|███▊ | 1632/2176 [00:28<00:09, 57.42it/s, loss=2.06e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 13:  87%|████▎| 1900/2176 [00:32<00:04, 58.47it/s, loss=2.06e+03, v_num=8]


Epoch 14:  75%|███▊ | 1632/2176 [00:29<00:09, 55.66it/s, loss=2.06e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 15:  75%|███▊ | 1632/2176 [00:27<00:09, 59.05it/s, loss=2.06e+03, v_num=8]
Validating: 0it [00:00, ?it/s]
Epoch 16:  43%|███    | 933/2176 [00:16<00:21, 57.80it/s, loss=2.1e+03, v_num=8]

# RL Training

## get the VAE


In [2]:
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete
import numpy as np
import yaml
from experiment import VAEXperiment
from models import *
import torch
from torchvision import transforms
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.1):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

#ideas
#maybe use wrapper to catch and save frames during RL model learning
#alternative is to use the model afterwards to generate frames during a test run
#
#??return mu, std or sample or just mu??
#
class VAE_ENC(ObservationWrapper):
    def __init__(self, env, vae, latent_dim,
                 mean=0,std=0.1,
                 size=(64,64)):
        super().__init__(env)
        #new obs space with std
        #self.observation_space = Box(shape=(2, latent_dim), low=-np.inf, high=np.inf)
        #just mean
        self.observation_space = Box(shape=(latent_dim,), low=-np.inf, high=np.inf)
        
        self.vae = vae
        #transforms
        self.mean = mean
        self.std = std
        self.size = size
        
        
        
    def observation(self, obs):
        #get frame
        #print(obs)
        frame = obs['pixels']#.to('cuda')
        #transform for VAE
        val_transforms = transforms.Compose([transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),
        AddGaussianNoise(self.mean, self.std),
        transforms.Resize(self.size),
        #transforms.Grayscale(),
        #transforms.Normalize(self.mean, self.std),
        ])
        frame = val_transforms(frame) #(c,h,w)
        frame = torch.unsqueeze(frame, 0)#.to(self.device) #make it (1,c,h,w)
        enc = self.vae.encode(frame)    
        enc = np.array([tensor.detach().cpu().numpy() for tensor in enc])
        #with std
        #enc = np.array([enc[0][0], enc[1][0]]) ## mu, std #  give only mu?
        #just mean
        enc = np.array(enc[0][0])
        return enc
    
def get_vae(version='version_0',log_directory='logs/BCE_sum_VAE/MSSIMVAE/'):

    model_path=log_directory+'/'+version+'/hparams.yaml'
    ckpt_path=log_directory+'/'+version+'/checkpoints/last.ckpt'

    config = yaml.safe_load(open(model_path))
    model = vae_models[config['model_params']['name']](**config['model_params'])
    ckpt = torch.load(ckpt_path)
    experiment = VAEXperiment(model, config['exp_params'])
    experiment.load_state_dict(ckpt['state_dict'])      
    vae = experiment.model
    return vae

In [3]:
vae = get_vae(version='version_9')

### make the env


In [4]:
env = gym.make("LunarLander-v2",
                continuous= True,
                gravity= -10.0,
                enable_wind= False,
                wind_power= 15.0,
                turbulence_power= 1.5,
                render_mode ='rgb_array')
seed = 42
env.reset(seed=seed)
env = PixelObservationWrapper(env)
env = VAE_ENC(env, vae, 16)
env = FrameStack(env, num_stack=2)
env = Monitor(env)

In [5]:
obs, info = env.reset()

In [6]:
env.observation_space


Box(-inf, inf, (2, 16), float32)

In [7]:
obs.shape

(2, 16)

In [8]:
obs

### PPO

In [9]:

agent_name = "vae_agent_5_PPO_latent_std"

env_name = "LunarLander-v2"

models_dir = f"RLmodels/"+env_name+'/'+agent_name
logdir = f"RLlogs/"+env_name+'/vae_PPO'

RL_train_steps = 1000000.0

callback = CheckpointCallback(save_freq=10000, save_path=models_dir) 

In [5]:
env = gym.make("LunarLander-v2",
                continuous= True,
                gravity= -10.0,
                enable_wind= False,
                wind_power= 15.0,
                turbulence_power= 1.5,
                render_mode ='rgb_array')
seed = 42
env.reset(seed=seed)
env = PixelObservationWrapper(env)
env = VAE_ENC(env, vae, 16)
env = FrameStack(env, num_stack=2)
env = Monitor(env)
#make_vec_env(env, n_envs=16, seed=seed)

In [11]:

agent = PPO('MlpPolicy',env,
    seed=seed,
    n_steps=1024,
    batch_size= 64,
    gae_lambda= 0.98,
    gamma= 0.999,
    n_epochs= 4,
    ent_coef=0.01,
    tensorboard_log = logdir,
    device="cuda")


In [12]:
agent.learn(total_timesteps=RL_train_steps, 
            callback=callback,
            tb_log_name=agent_name)

In [ ]:
agent

In [ ]:
agent = PPO.load('RLmodels/LunarLander-v2/vae_agent_0_PPO/rl_model_593590_steps.zip')

#### reload old model continue training


In [5]:

agent_name = "vae_agent_4_PPO_latent_std"

env_name = "LunarLander-v2"

models_dir = f"RLmodels/"+env_name+'/'+agent_name
logdir = f"RLlogs/"+env_name+'/vae_PPO'

RL_train_steps = 420000.0#1000000.0

callback = CheckpointCallback(save_freq=10000, save_path=models_dir) 

# agent = PPO('MlpPolicy',env,
#     seed=seed,
#     n_steps=1024,
#     batch_size= 64,
#     gae_lambda= 0.98,
#     gamma= 0.999,
#     n_epochs= 4,
#     ent_coef=0.01,
#     tensorboard_log = logdir)

agent = PPO.load('RLmodels/LunarLander-v2/vae_agent_4_PPO_latent_std/rl_model_530000_steps.zip', env=env)
print(agent.get_env())
agent.learn(total_timesteps=RL_train_steps, reset_num_timesteps=False, 
            callback=callback,
            tb_log_name=agent_name)

KeyboardInterrupt: 

In [6]:

agent_name = "vae_agent_4_PPO"

env_name = "LunarLander-v2"

models_dir = f"RLmodels/"+env_name+'/'+agent_name
logdir = f"RLlogs/"+env_name+'/vae_PPO'

RL_train_steps = 440000

callback = CheckpointCallback(save_freq=10000, save_path=models_dir) 

# agent = PPO('MlpPolicy',env,
#     seed=seed,
#     n_steps=1024,
#     batch_size= 64,
#     gae_lambda= 0.98,
#     gamma= 0.999,
#     n_epochs= 4,
#     ent_coef=0.01,
#     tensorboard_log = logdir)

agent = PPO.load('RLmodels/LunarLander-v2/vae_agent_4_PPO/rl_model_560000_steps.zip', env=env)
print(agent.get_env())
agent.learn(total_timesteps=RL_train_steps, reset_num_timesteps=False, 
            callback=callback,
            tb_log_name=agent_name)

### SAC

In [ ]:
  n_timesteps=float 5e5,
  policy='MlpPolicy',
  batch_size= 256,
  learning_rate= 'lin_7.3e-4',
  buffer_size= 1000000,
  ent_coef= 'auto',
  gamma= 0.99,
  tau=0.01,
  train_freq= 1,
  gradient_steps= 1,
  learning_starts= 10000,
  policy_kwargs= dict(net_arch=[400, 300]),

In [ ]:
from typing import Callable

def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func


models_dir = f"RLmodels/LunarLander-v2_first_run_l10/"
logdir = f"RLlogs/LunarLander-v2_first_run_l10/"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
if not os.path.exists(logdir):
    os.makedirs(logdir)

    
total_timesteps =  500000  # Total number of timesteps for training
initial_lr = 7.3e-4  # Initial learning rate

# Define the learning rate schedule
lr_schedule = linear_schedule(initial_lr)
    
    
model = SAC(   
    policy='MlpPolicy',
    env=env,
    seed=seed,
    batch_size= 256,
    learning_rate= lr_schedule,
    buffer_size= 1000000,
    ent_coef= 'auto',
    gamma= 0.99,
    tau=0.01,
    train_freq= 1,
    gradient_steps= 1,
    learning_starts= 10000,
    policy_kwargs= dict(net_arch=[400, 300]),
    use_sde= True,
    tensorboard_log=logdir)

In [ ]:
iteration=3
models_dir = f"RLmodels/LunarLander-v2/"+str(iteration)
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
agent_name = "SAC"



In [ ]:
iteration=3
agent_name = "lunarlander_thirdVAE_l10_SAC"
callback = CheckpointCallback(save_freq=10000, save_path=models_dir+str(iteration))   
#train the RL agent
model.learn(total_timesteps=total_timesteps,callback=callback,tb_log_name=agent_name)

In [ ]:
agent = model

## collect more images with RL agent

In [26]:

agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_4_PPO/rl_model_520000_steps.zip')




In [27]:
save_path='Data/lunar-lander/vae_PPO_4_rl_model'
data_name='vae_PPO_4_rl_model_520000_'
num_of_episodes=100
i=0
for episode in range(num_of_episodes):
    observation, info = env.reset()
    done = False
    while not done: 
        action, _states = agent.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        #state, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        if terminated:
            done = True
        if truncated:
            done = True
        current_frame = env.render()

        i+=1
        im = Image.fromarray(np.array(current_frame))
        im.save(save_path+'/'+data_name+'_'+str(i)+'.jpeg')
        im.save('Data/lunar-lander/data_comb_third_vae/'+data_name+'_'+str(i)+'.jpeg')

In [6]:
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_4_PPO/rl_model_470000_steps.zip')
save_path='Data/lunar-lander/vae_PPO_5_rl_model'
data_name='vae_PPO_5_rl_model_470000_'
num_of_episodes=100
i=0
for episode in range(num_of_episodes):
    observation, info = env.reset()
    done = False
    while not done: 
        action, _states = agent.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        #state, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        if terminated:
            done = True
        if truncated:
            done = True
        current_frame = env.render()

        i+=1
        im = Image.fromarray(np.array(current_frame))
        im.save(save_path+'/'+data_name+'_'+str(i)+'.jpeg')
        im.save('Data/lunar-lander/data_comb_third_vae/'+data_name+'_'+str(i)+'.jpeg')

In [8]:
model_list=['270000','300000','350000','430000','450000','530000','530000','540000',
           '560000','560000','560000','620000','780000','860000','900000','1000000']
for model_steps in model_list:
    print(model_steps)
    agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_5_PPO/rl_model_'+model_steps+'_steps.zip')
    save_path='Data/lunar-lander/vae_PPO_6_rl_model'
    data_name='vae_PPO_6_rl_model_'+model_steps+'_'
    num_of_episodes=100
    i=0
    for episode in range(num_of_episodes):
        observation, info = env.reset()
        done = False
        while not done: 
            action, _states = agent.predict(observation, deterministic=True)
            observation, reward, terminated, truncated, info = env.step(action)
            #state, reward, terminated, truncated, info = env.step(action)
            #print(terminated, truncated)
            if terminated:
                done = True
            if truncated:
                done = True
            current_frame = env.render()

            i+=1
            im = Image.fromarray(np.array(current_frame))
            im.save(save_path+'/'+data_name+'_'+str(i)+'.jpeg')
            im.save('Data/lunar-lander/data_comb_third_vae/'+data_name+'_'+str(i)+'.jpeg')

270000
300000
350000
430000
450000
530000
530000
540000
560000
560000
560000
620000
780000
860000
900000
1000000


In [7]:
def evalu(Paths, n_episodes, rounds, env):
    results =[]
    for path in Paths:
        results.append(path)
        print(path)
        for i in range(rounds):
            
            agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_5_PPO/'+path)
            model = agent
            mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_episodes)
            print(mean_reward, std_reward)
            results.append([mean_reward, std_reward])
    return results

In [16]:
#fifth train run checks
paths=['rl_model_310000_steps.zip','rl_model_350000_steps.zip','rl_model_420000_steps.zip','rl_model_470000_steps.zip',
       'rl_model_480000_steps.zip','rl_model_540000_steps.zip','rl_model_620000_steps.zip','rl_model_630000_steps.zip',
       'rl_model_750000_steps.zip','rl_model_800000_steps.zip','rl_model_820000_steps.zip','rl_model_990000_steps.zip',
       'rl_model_1000000_steps.zip']

results = evalu(paths, 50, 6, env)

rl_model_310000_steps.zip
-116.00189984 48.173261325602454
-147.22103584 93.28058845311143
-122.3601518 68.48546254138576
-109.65156370000001 45.98363277798451
-119.97249407999999 59.133286944700295
-120.90168293999999 60.521578946248276
rl_model_350000_steps.zip
-140.87900372000001 116.34292248150747
-152.78205192000001 170.41982807154224
-133.00414356 139.03231799778627
-127.52937618 108.82996012410217
-174.48758707999997 184.70879321929263
-127.44506082 103.4179319667611
rl_model_420000_steps.zip
-101.10374098000003 48.991455887470345
-125.2366053 61.19376644712187
-115.62237732 68.14981505303982
-115.75059871999999 52.821401090185944
-110.50489874 49.302187038917666
-117.86656962 64.9035439751879
rl_model_470000_steps.zip
-109.99415306 79.00400064841807
-115.72262010000003 57.97817411929974
-135.39715248000002 101.77474507983325
-113.44919449999999 74.17477932462221
-125.76616807999997 65.98659833807967
-113.04046179999999 65.3458294797091
rl_model_480000_steps.zip
-125.24092057999

In [ ]:
#sixth train run checks
#paths=['rl_model_270000_steps.zip','rl_model_300000_steps.zip','rl_model_350000_steps.zip','rl_model_430000_steps.zip',
       #'rl_model_450000_steps.zip','rl_model_530000_steps.zip','rl_model_540000_steps.zip','rl_model_560000_steps.zip',
paths=['rl_model_540000_steps.zip','rl_model_560000_steps.zip','rl_model_620000_steps.zip','rl_model_720000_steps.zip',
       'rl_model_760000_steps.zip','rl_model_900000_steps.zip',
       'rl_model_950000_steps.zip','rl_model_1000000_steps.zip']

results = evalu(paths, 50, 6, env)

rl_model_270000_steps.zip
-101.7188162 34.622291599354064
-120.71703412000001 72.63509005466624
-115.69102425999999 102.86393066773995
-113.49202497999998 69.75705627434651
-108.11709656 51.651803446811684
-99.65894 60.023901223715036
rl_model_300000_steps.zip
-108.17870642 49.41617326841123
-111.57683195999999 75.30008297456088
-96.34163336000002 55.54672127664605
-95.53988695999999 47.14636729439679
-118.78156768 62.054477193236025
-112.75843323999999 61.0895515187932
rl_model_350000_steps.zip
-138.29050972 110.46009696877566
-110.68535132 79.92574268589397
-97.19806898 75.63032803101947
-114.87606916 79.21008599244246
-117.74320352000001 83.63605925453615
-117.92392201999999 77.80348942624741
rl_model_430000_steps.zip
-104.32817816000001 66.34849633545784
-104.32678632000001 101.83977172094458
-121.34148209999998 89.0181966767491
-117.00438529999998 126.51472740756468
-99.23385462 79.65321486833281
-109.63684122000001 73.94727344242166
rl_model_450000_steps.zip
-121.35729966 103.881

In [6]:
model_list=['270000','300000','350000','430000','450000','530000','530000','540000',
           '560000','560000','560000','620000','780000','860000','900000','1000000']

In [8]:
paths=['rl_model_540000_steps.zip','rl_model_560000_steps.zip','rl_model_620000_steps.zip','rl_model_720000_steps.zip',
       'rl_model_760000_steps.zip','rl_model_900000_steps.zip',
       'rl_model_950000_steps.zip','rl_model_1000000_steps.zip']
results = evalu(paths, 50, 6, env)

rl_model_540000_steps.zip
-114.13175486 96.83830006965592
-100.52232648 83.73023829476172
-103.75459839999999 118.86707794871532
-131.27673964000002 125.5768900514361
-119.09889191999999 95.70111925394048
-119.03784950000001 104.43449039252603
rl_model_560000_steps.zip
-124.52374326 104.36136824488354
-106.52858496 80.79312520389392
-131.46363722 106.99031236696183
-133.53370852 115.74515892751428
-122.6102376 100.2138100322701
-80.40005846 78.71738852868411
rl_model_620000_steps.zip
-116.19128807999999 104.95152899920339
-130.60956076 88.53216399631891
-111.0474926 69.62661566003538
-125.5737587 102.77854949720674
-128.99222565999997 76.69050962248674
-113.98169139999999 91.84568637146326
rl_model_720000_steps.zip
-139.3994035 114.33626529320556
-142.89662467999997 200.7220619412855
-171.56960314 142.9017031753909
-155.07969563999998 132.90848078904227
-153.3711911 131.59169271564664
-114.91679566 98.6651243500895
rl_model_760000_steps.zip
-164.45051180000002 154.3061972786374
-169.04

In [15]:
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_320000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_320000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)

-96.49359340999997 57.95539504263404
-107.61280061 57.51255209237989


In [16]:
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_470000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_470000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)

-138.17623149999997 107.63718796831819
-122.54357180000001 108.02337386761968


In [17]:
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_520000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_520000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)

-107.52073599 93.97499762150377
-112.74787249 92.60049502700184


In [18]:
#best
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_540000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_540000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)

-114.81666070000001 85.92475239119177
-127.86045111 84.52887222655937


In [20]:
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_630000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_630000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)

-147.48387839999998 116.52575973373199
-142.16113754 103.94095785982186


In [22]:
agent= PPO.load('RLmodels/LunarLander-v2/vae_agent_3_PPO/rl_model_990000_steps.zip')
model = agent
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)
print(mean_reward, std_reward)

-178.25446521 132.68786986232666


In [ ]:



images = []
obs = model.env.reset()
img = model.env.render(mode="rgb_array")
for i in range(450):
    images.append(img)
    action, _ = model.predict(obs)
    obs, _, _ ,_ = model.env.step(action)
    img = model.env.render(mode="rgb_array")

imgs = [Image.fromarray(img) for img in images]
# duration is the number of milliseconds between frames; this is 40 frames per second
imgs[0].save("lunar_lander_sac_l10_v1.gif", save_all=True, append_images=imgs[1:], duration=50, loop=0)

In [ ]:
model_folder = 'RLmodels/LunarLander-v2/11'
# Get a list of all files in the directory
files = os.listdir(model_folder)
# Sort the files based on modification time (newest first)
files.sort(key=lambda x: os.path.getmtime(os.path.join(model_folder, x)))


In [ ]:
model0 = SAC.load(model_folder+'/'+files[0], env=env)

In [ ]:
num_of_imgs = 6377

#collect new images with trained RL agent
#aprox double number of images to train the vae on

num_of_episodes = 2

target_num_of_imgs = num_of_imgs*2
#get ~ enough imgs, but its done via episodes.. so if the model becomes successful it would be way less imgs    
num_of_imgs += collect_imgs(num_of_episodes=6, num_of_previous_imgs = num_of_imgs,
            data_dir="Data/",data_name = "img_Data",
            model=model) 



model_folder = 'RLmodels/LunarLander-v2/11'
# Get a list of all files in the directory
files = os.listdir(model_folder)
files.sort(key=lambda x: os.path.getmtime(os.path.join(model_folder, x)))
for file in files:   
    model0 = SAC.load(model_folder+'/'+file, env=env)

    num_of_imgs += collect_imgs(num_of_episodes=num_of_episodes,num_of_previous_imgs = num_of_imgs,
            data_dir="Data/",data_name = "lunar_Data",
            model=model0) 

In [ ]:
import os
import torch
from torch import Tensor
from pathlib import Path
from typing import List, Optional, Sequence, Union, Any, Callable
from torchvision.datasets.folder import default_loader
from pytorch_lightning import LightningDataModule
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
data_dir = Path('Data/lunar-lander/vae_PPO_second_rl_model_850000') # relative path to directory + filename        
imgs = sorted([f for f in data_dir.iterdir() if f.suffix == '.jpeg'])
print('MyDataset: ', len(imgs), ' images found')
